In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import itertools
import warnings
import holidays
import mlflow
import dagshub

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Import from src directory
from src.data_utils import load_and_process_taxi_data, transform_raw_data_into_ts_data

# Initialize MLflow tracking
dagshub.init(repo_owner="gourimenon8", repo_name="sp25_taxi", mlflow=True)
mlflow.set_experiment("improved_arma_model")

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("deep")
plt.rcParams['figure.figsize'] = (12, 6)

def load_data():
    """Load and process NYC taxi data"""
    print("Loading data...")
    rides = load_and_process_taxi_data(year=2023)
    # rides = pd.concat([rides1, rides2], ignore_index=True)
    
    # Filter for location ID 43
    LOCATION_ID = 43
    print(f"Filtering data for location ID {LOCATION_ID}...")
    temp_rides = rides[rides["pickup_location_id"] == LOCATION_ID]
    
    # Transform into time series data
    ts_data = transform_raw_data_into_ts_data(temp_rides)
    ts_data = ts_data.drop(columns=["pickup_location_id"])
    
    # Convert pickup_hour to datetime and set as index
    ts_data["pickup_hour"] = pd.to_datetime(ts_data["pickup_hour"])
    ts_data.set_index("pickup_hour", inplace=True)
    
    # Ensure data is in time series format
    ts_data = ts_data.asfreq("H")
    ts_data["rides"] = ts_data["rides"].astype(np.float32)
    
    print(f"Time series data shape: {ts_data.shape}")
    print(ts_data.head())
    
    return ts_data

def add_time_features(ts_data):
    """Add time-based features to the data"""
    time_features = pd.DataFrame(index=ts_data.index)
    
    # Extract time components
    time_features['hour'] = ts_data.index.hour
    time_features['day_of_week'] = ts_data.index.dayofweek
    time_features['day'] = ts_data.index.day
    time_features['month'] = ts_data.index.month
    time_features['quarter'] = ts_data.index.quarter
    time_features['year'] = ts_data.index.year
    
    # Create binary features
    time_features['is_weekend'] = (time_features['day_of_week'] >= 5).astype(int)
    time_features['is_rush_hour_am'] = ((time_features['hour'] >= 7) & 
                                        (time_features['hour'] <= 9)).astype(int)
    time_features['is_rush_hour_pm'] = ((time_features['hour'] >= 16) & 
                                        (time_features['hour'] <= 19)).astype(int)
    
    # Add US holidays
    us_holidays = holidays.US(years=[2022, 2023])
    time_features['is_holiday'] = [1 if d.date() in us_holidays else 0 for d in time_features.index]
    
    # Merge with original data
    result = pd.concat([ts_data, time_features], axis=1)
    return result

def check_stationarity(series, window=24):
    """
    Check stationarity using rolling statistics and Augmented Dickey-Fuller test
    
    Parameters:
    -----------
    series : pandas.Series
        Time series to check for stationarity
    window : int
        Window size for rolling statistics
        
    Returns:
    --------
    bool
        True if stationary, False otherwise
    dict
        ADF test results
    """
    # Calculate rolling statistics
    rolling_mean = series.rolling(window=window).mean()
    rolling_std = series.rolling(window=window).std()
    
    # Plot rolling statistics
    plt.figure(figsize=(12, 6))
    plt.subplot(211)
    plt.title('Rolling Mean & Standard Deviation')
    plt.plot(series, label='Original')
    plt.plot(rolling_mean, label='Rolling Mean')
    plt.plot(rolling_std, label='Rolling Std')
    plt.legend()
    
    # Perform ADF test
    adf_result = adfuller(series.dropna())
    adf_output = {
        'Test Statistic': adf_result[0],
        'p-value': adf_result[1],
        '#Lags Used': adf_result[2],
        'Number of Observations': adf_result[3],
        'Critical Values': adf_result[4]
    }
    
    # Display ADF test results
    plt.subplot(212)
    plt.title('Augmented Dickey-Fuller Test Results')
    plt.axis('off')
    result_text = '\n'.join([f"{key}: {value}" if not isinstance(value, dict) else f"{key}: {value['1%']}, {value['5%']}, {value['10%']}" 
                            for key, value in adf_output.items()])
    plt.text(0.01, 0.8, result_text, fontsize=12)
    plt.tight_layout()
    plt.savefig("stationarity_check.png")
    
    # Print results
    print('\nResults of Augmented Dickey-Fuller Test:')
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    
    # Return stationarity boolean and ADF results
    is_stationary = adf_result[1] <= 0.05
    print(f'Series is{"" if is_stationary else " not"} stationary')
    
    return is_stationary, adf_output

def plot_time_series(ts_data, exog_data=None):
    """Plot time series data and derived features"""
    # Plot time series
    plt.figure(figsize=(14, 8))
    plt.subplot(3, 1, 1)
    plt.plot(ts_data.index, ts_data['rides'])
    plt.title('NYC Taxi Rides Time Series (Location ID 43)')
    plt.ylabel('Number of Rides')
    
    # Plot hourly pattern
    plt.subplot(3, 1, 2)
    hourly_avg = ts_data.groupby(ts_data.index.hour)['rides'].mean()
    plt.bar(hourly_avg.index, hourly_avg.values)
    plt.title('Average Rides by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Average Rides')
    
    # Plot weekly pattern
    plt.subplot(3, 1, 3)
    daily_avg = ts_data.groupby(ts_data.index.dayofweek)['rides'].mean()
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    plt.bar([day_names[i] for i in daily_avg.index], daily_avg.values)
    plt.title('Average Rides by Day of Week')
    plt.xlabel('Day of Week')
    plt.ylabel('Average Rides')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig("time_series_patterns.png")
    
    # If exogenous variables are provided, plot their relationship
    if exog_data is not None and not exog_data.empty:
        plt.figure(figsize=(15, 10))
        
        # Plot relationship with hour
        plt.subplot(2, 2, 1)
        hour_avg = pd.DataFrame({'hour': exog_data['hour'], 'rides': ts_data['rides']})
        hour_avg = hour_avg.groupby('hour')['rides'].mean()
        plt.bar(hour_avg.index, hour_avg.values)
        plt.title('Average Rides by Hour')
        
        # Plot relationship with day of week
        plt.subplot(2, 2, 2)
        dow_avg = pd.DataFrame({'day_of_week': exog_data['day_of_week'], 'rides': ts_data['rides']})
        dow_avg = dow_avg.groupby('day_of_week')['rides'].mean()
        plt.bar([day_names[i] for i in dow_avg.index], dow_avg.values)
        plt.title('Average Rides by Day of Week')
        plt.xticks(rotation=45)
        
        # Plot relationship with month
        plt.subplot(2, 2, 3)
        month_avg = pd.DataFrame({'month': exog_data['month'], 'rides': ts_data['rides']})
        month_avg = month_avg.groupby('month')['rides'].mean()
        month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        plt.bar([month_names[i-1] for i in month_avg.index], month_avg.values)
        plt.title('Average Rides by Month')
        
        # Plot relationship with holidays
        plt.subplot(2, 2, 4)
        holiday_avg = pd.DataFrame({'is_holiday': exog_data['is_holiday'], 'rides': ts_data['rides']})
        holiday_avg = holiday_avg.groupby('is_holiday')['rides'].mean()
        plt.bar(['Non-Holiday', 'Holiday'], holiday_avg.values)
        plt.title('Average Rides on Holidays vs. Non-Holidays')
        
        plt.tight_layout()
        plt.savefig("feature_relationships.png")

def analyze_acf_pacf(series, lags=48):
    """Analyze and plot ACF and PACF to help identify ARMA orders"""
    plt.figure(figsize=(12, 8))
    
    plt.subplot(211)
    plot_acf(series.dropna(), ax=plt.gca(), lags=lags)
    plt.title('Autocorrelation Function (ACF)')
    
    plt.subplot(212)
    plot_pacf(series.dropna(), ax=plt.gca(), lags=lags)
    plt.title('Partial Autocorrelation Function (PACF)')
    
    plt.tight_layout()
    plt.savefig("acf_pacf_plots.png")
    
    # Get significant lags from ACF and PACF
    acf_values = acf(series.dropna(), nlags=lags)
    pacf_values = pacf(series.dropna(), nlags=lags)
    
    # Use 95% confidence interval (1.96/sqrt(n))
    confidence_interval = 1.96 / np.sqrt(len(series.dropna()))
    
    # Find significant lags
    significant_acf_lags = [i for i, val in enumerate(acf_values) if abs(val) > confidence_interval and i > 0]
    significant_pacf_lags = [i for i, val in enumerate(pacf_values) if abs(val) > confidence_interval and i > 0]
    
    print(f"Significant ACF lags: {significant_acf_lags[:10]}...")
    print(f"Significant PACF lags: {significant_pacf_lags[:10]}...")
    
    # Suggest ARMA orders based on ACF and PACF
    print("\nSuggested ARMA orders based on ACF and PACF patterns:")
    print(f"AR(p) order suggestions: {[1, 2, 3, 24]} (from PACF)")
    print(f"MA(q) order suggestions: {[1, 2, 3, 24]} (from ACF)")
    
    return significant_acf_lags, significant_pacf_lags

def create_train_test_sets(ts_data, test_size=0.2):
    """Split data into training and testing sets"""
    train_size = int(len(ts_data) * (1 - test_size))
    train = ts_data.iloc[:train_size]
    test = ts_data.iloc[train_size:]
    print(f"Training set size: {train.shape}")
    print(f"Testing set size: {test.shape}")
    return train, test

def seasonal_decompose(series):
    """Perform seasonal decomposition of time series"""
    decomposition = sm.tsa.seasonal_decompose(series, model='additive', period=24)
    
    plt.figure(figsize=(12, 10))
    plt.subplot(411)
    plt.plot(decomposition.observed)
    plt.title('Observed')
    
    plt.subplot(412)
    plt.plot(decomposition.trend)
    plt.title('Trend')
    
    plt.subplot(413)
    plt.plot(decomposition.seasonal)
    plt.title('Seasonality')
    
    plt.subplot(414)
    plt.plot(decomposition.resid)
    plt.title('Residuals')
    
    plt.tight_layout()
    plt.savefig("seasonal_decomposition.png")
    
    return decomposition

def find_best_arma_orders(train_data, max_p=5, max_q=5, exog=None):
    """Find the best ARMA orders by fitting multiple models"""
    best_aic = float("inf")
    best_order = None
    results = []
    
    print("\nFinding best ARMA parameters...")
    for p, q in itertools.product(range(max_p + 1), range(max_q + 1)):
        if p == 0 and q == 0:
            continue
        
        try:
            model = ARIMA(train_data, order=(p, 0, q), exog=exog)
            model_fit = model.fit()
            aic = model_fit.aic
            results.append((p, q, aic))
            
            if aic < best_aic:
                best_aic = aic
                best_order = (p, q)
                
            print(f"ARMA({p}, {q}) - AIC: {aic:.3f}")
        except:
            continue
    
    # Sort results by AIC
    results.sort(key=lambda x: x[2])
    
    print("\nBest 5 ARMA Models:")
    for p, q, aic in results[:5]:
        print(f"ARMA({p}, {q}) - AIC: {aic:.3f}")
    
    print(f"\nBest ARMA Order: {best_order}")
    
    return best_order, results

def fit_arma_model(train_data, order, exog=None):
    """Fit ARMA model with specified order"""
    model = ARIMA(train_data, order=(order[0], 0, order[1]), exog=exog)
    model_fit = model.fit()
    print(model_fit.summary())
    return model_fit

def evaluate_forecast(actual, forecast, model_name="ARMA"):
    """Evaluate forecast using various metrics"""
    mae = mean_absolute_error(actual, forecast)
    rmse = np.sqrt(mean_squared_error(actual, forecast))
    r2 = r2_score(actual, forecast)
    mape = np.mean(np.abs((actual - forecast) / actual)) * 100
    
    # Calculate directional accuracy
    directions_actual = np.sign(np.diff(np.array(actual)))
    directions_forecast = np.sign(np.diff(np.array(forecast)))
    directional_accuracy = np.mean(directions_actual == directions_forecast) * 100
    
    print(f"\n{model_name} Forecast Evaluation:")
    print(f"Mean Absolute Error (MAE): {mae:.3f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.3f}%")
    print(f"R-squared (R²): {r2:.3f}")
    print(f"Directional Accuracy: {directional_accuracy:.2f}%")
    
    # Plot forecast vs actual
    plt.figure(figsize=(12, 6))
    plt.plot(actual.index, actual, label='Actual')
    plt.plot(actual.index, forecast, label='Forecast', color='red')
    plt.title(f'{model_name} - Forecast vs Actual')
    plt.xlabel('Date')
    plt.ylabel('Number of Rides')
    plt.legend()
    plt.savefig(f"{model_name.lower().replace(' ', '_')}_forecast.png")
    
    # Plot forecast error
    plt.figure(figsize=(12, 6))
    plt.plot(actual.index, actual - forecast, label='Forecast Error')
    plt.axhline(y=0, color='r', linestyle='--')
    plt.title(f'{model_name} - Forecast Error')
    plt.xlabel('Date')
    plt.ylabel('Error (Actual - Forecast)')
    plt.legend()
    plt.savefig(f"{model_name.lower().replace(' ', '_')}_error.png")
    
    # Return metrics as a dictionary
    return {
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'r2': r2,
        'directional_accuracy': directional_accuracy
    }

def analyze_residuals(model_fit, model_name="ARMA"):
    """Analyze residuals of a fitted model"""
    residuals = model_fit.resid
    
    # Plot residuals
    plt.figure(figsize=(12, 8))
    
    # Plot residuals over time
    plt.subplot(311)
    plt.plot(residuals)
    plt.title(f'{model_name} Model - Residuals Over Time')
    plt.xlabel('Date')
    plt.ylabel('Residual')
    plt.axhline(y=0, color='r', linestyle='--')
    
    # Histogram of residuals
    plt.subplot(312)
    sns.histplot(residuals, kde=True)
    plt.title('Histogram of Residuals')
    plt.xlabel('Residual Value')
    
    # Q-Q plot
    plt.subplot(313)
    sm.qqplot(residuals, line='s', ax=plt.gca())
    plt.title('Q-Q Plot of Residuals')
    
    plt.tight_layout()
    plt.savefig(f"{model_name.lower().replace(' ', '_')}_residuals.png")
    
    # ACF of residuals to check if they are white noise
    plt.figure(figsize=(12, 4))
    plot_acf(residuals.dropna(), ax=plt.gca(), lags=40)
    plt.title(f'ACF of {model_name} Model Residuals')
    plt.savefig(f"{model_name.lower().replace(' ', '_')}_residuals_acf.png")
    
    # Ljung-Box test
    lb_test = sm.stats.acorr_ljungbox(residuals, lags=[10, 20, 30], return_df=True)
    print(f"\nLjung-Box Test Results for {model_name} Model Residuals:")
    print(lb_test)
    
    # Check normality of residuals with Jarque-Bera test
    jb_test = sm.stats.jarque_bera(residuals.dropna())
    print(f"\nJarque-Bera Test Results for {model_name} Model Residuals:")
    print(f"Statistic: {jb_test[0]:.3f}")
    print(f"p-value: {jb_test[1]:.3f}")
    if jb_test[1] < 0.05:
        print("Residuals are not normally distributed (p < 0.05)")
    else:
        print("Residuals appear to be normally distributed (p >= 0.05)")
    
    # Descriptive statistics of residuals
    residuals_stats = residuals.describe()
    print("\nResiduals Descriptive Statistics:")
    print(residuals_stats)
    
    return residuals, jb_test, lb_test

def forecast_future(model_fit, steps=24, exog=None, plot=True):
    """Forecast future values using fitted model"""
    forecast = model_fit.forecast(steps=steps, exog=exog)
    
    # Create date index for forecast
    last_date = model_fit.data.index[-1]
    forecast_index = pd.date_range(start=last_date, periods=steps + 1, freq="H")[1:]
    
    if plot:
        # Plot historical data and forecast
        plt.figure(figsize=(12, 6))
        plt.plot(model_fit.data.index, model_fit.data.endog, label='Historical Data')
        plt.plot(forecast_index, forecast, label='Forecast', color='red')
        plt.title('ARMA Model Forecast')
        plt.xlabel('Date')
        plt.ylabel('Number of Rides')
        plt.legend()
        plt.savefig("arma_forecast.png")
    
    # Create forecast DataFrame
    forecast_df = pd.DataFrame({
        'forecast': forecast
    }, index=forecast_index)
    
    return forecast_df

def main():
    # Load and prepare data
    ts_data = load_data()
    
    # Add time-based features
    ts_with_features = add_time_features(ts_data)
    
    # Visualize time series data and patterns
    plot_time_series(ts_data, ts_with_features)
    
    # Check stationarity
    is_stationary, adf_results = check_stationarity(ts_data['rides'])
    
    # Analyze ACF and PACF to identify potential AR and MA orders
    acf_lags, pacf_lags = analyze_acf_pacf(ts_data['rides'])
    
    # Perform seasonal decomposition
    decomposition = seasonal_decompose(ts_data['rides'])
    
    # Split data into training and testing sets
    train_data, test_data = create_train_test_sets(ts_data)
    train_features, test_features = create_train_test_sets(ts_with_features)
    
    # Extract exogenous variables
    exog_columns = ['hour', 'day_of_week', 'is_weekend', 'is_holiday', 
                    'is_rush_hour_am', 'is_rush_hour_pm']
    train_exog = train_features[exog_columns] if exog_columns else None
    test_exog = test_features[exog_columns] if exog_columns else None
    
    # Find best ARMA orders automatically
    auto_best_order, auto_results = find_best_arma_orders(train_data['rides'], max_p=5, max_q=5)
    
    # Fit best ARMA model (without exogenous variables)
    best_arma_model = fit_arma_model(train_data['rides'], auto_best_order)
    
    # Forecast and evaluate
    test_forecast = best_arma_model.forecast(steps=len(test_data))
    metrics_arma = evaluate_forecast(test_data['rides'], test_forecast, model_name="ARMA")
    
    # Analyze residuals
    residuals_arma, jb_test_arma, lb_test_arma = analyze_residuals(best_arma_model, model_name="ARMA")
    
    # Fit ARMA model with exogenous variables (ARMAX)
    try:
        print("\nFitting ARMAX model with exogenous variables...")
        armax_model = fit_arma_model(train_data['rides'], auto_best_order, exog=train_exog)
        
        # Forecast and evaluate ARMAX
        armax_forecast = armax_model.forecast(steps=len(test_data), exog=test_exog)
        metrics_armax = evaluate_forecast(test_data['rides'], armax_forecast, model_name="ARMAX")
        
        # Analyze residuals for ARMAX
        residuals_armax, jb_test_armax, lb_test_armax = analyze_residuals(armax_model, model_name="ARMAX")
        
        # Compare models
        print("\nModel Comparison:")
        print(f"ARMA MAE: {metrics_arma['mae']:.3f}, RMSE: {metrics_arma['rmse']:.3f}, R²: {metrics_arma['r2']:.3f}")
        print(f"ARMAX MAE: {metrics_armax['mae']:.3f}, RMSE: {metrics_armax['rmse']:.3f}, R²: {metrics_armax['r2']:.3f}")
        
        # Determine best model
        best_model = "ARMAX" if metrics_armax['mae'] < metrics_arma['mae'] else "ARMA"
        best_metrics = metrics_armax if best_model == "ARMAX" else metrics_arma
        print(f"\nBest model based on MAE: {best_model}")
        
        # Create bar chart comparing metrics
        plt.figure(figsize=(12, 6))
        models = ['ARMA', 'ARMAX']
        mae_values = [metrics_arma['mae'], metrics_armax['mae']]
        rmse_values = [metrics_arma['rmse'], metrics_armax['rmse']]
        r2_values = [metrics_arma['r2'], metrics_armax['r2']]
        
        x = np.arange(len(models))
        width = 0.25
        
        plt.bar(x - width, mae_values, width, label='MAE')
        plt.bar(x, rmse_values, width, label='RMSE')
        plt.bar(x + width, r2_values, width, label='R²')
        
        plt.xlabel('Models')
        plt.ylabel('Metric Value')
        plt.title('Model Performance Comparison')
        plt.xticks(x, models)
        plt.legend()
        plt.savefig("model_comparison.png")
        
    except Exception as e:
        print(f"Error fitting ARMAX model: {e}")
        best_model = "ARMA"
        best_metrics = metrics_arma
    
    # Train final model on all data
    print("\nTraining final model on all data...")
    if best_model == "ARMA":
        final_model = fit_arma_model(ts_data['rides'], auto_best_order)
    else:
        final_model = fit_arma_model(ts_data['rides'], auto_best_order, exog=ts_with_features[exog_columns])
    
    # Forecast future values
    future_steps = 48  # 2 days
    future_forecast = forecast_future(final_model, steps=future_steps)
    
    print(f"\nForecasted values for the next {future_steps} hours:")
    print(future_forecast.head(10))
    
    # Log to MLflow
    with mlflow.start_run():
        # Log parameters
        mlflow.log_param("best_model_type", best_model)
        mlflow.log_param("arma_order", auto_best_order)
        if best_model == "ARMAX":
            mlflow.log_param("exog_variables", exog_columns)
        
        # Log metrics
        for metric_name, metric_value in best_metrics.items():
            mlflow.log_metric(metric_name, metric_value)
        
        # Log artifacts
        mlflow.log_artifact("time_series_patterns.png")
        mlflow.log_artifact("stationarity_check.png")
        mlflow.log_artifact("acf_pacf_plots.png")
        mlflow.log_artifact("seasonal_decomposition.png")
        mlflow.log_artifact(f"{best_model.lower()}_forecast.png")
        mlflow.log_artifact(f"{best_model.lower()}_error.png")
        mlflow.log_artifact(f"{best_model.lower()}_residuals.png")
        mlflow.log_artifact("arma_forecast.png")
        if best_model == "ARMAX":
            mlflow.log_artifact("feature_relationships.png")
            mlflow.log_artifact("model_comparison.png")
    
    print("\nARMA modeling complete!")

if __name__ == "__main__":
    main()

Accessing as gourimenon8

Initialized MLflow to track repo "gourimenon8/sp25_taxi"

Repository gourimenon8/sp25_taxi initialized!

Loading data...
File already exists for 2023-01.
Loading data for 2023-01...
Total records: 3,066,766
Valid records: 2,993,140
Records dropped: 73,626 (2.40%)
Successfully processed data for 2023-01.
File already exists for 2023-02.
Loading data for 2023-02...
Total records: 2,913,955
Valid records: 2,845,058
Records dropped: 68,897 (2.36%)
Successfully processed data for 2023-02.
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 3,403,766
Valid records: 3,331,705
Records dropped: 72,061 (2.12%)
Successfully processed data for 2023-03.
File already exists for 2023-04.
Loading data for 2023-04...
Total records: 3,288,250
Valid records: 3,214,922
Records dropped: 73,328 (2.23%)
Successfully processed data for 2023-04.
File already exists for 2023-05.
Loading data for 2023-05...
Total records: 3,513,649
Valid records: 3,435,875
Records dropped: 77,774 (2.21%)
Successfully processed data for 2023-05.
File already exists for 2023-06.
Loading data for 2023-06...
Tot